In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rectal_Cancer/GSE108166'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differential clinicopathological and molecular features within late-onset colorectal cancer according to tumor location"
!Series_summary	"Background: Since there is a predilection of some clinical and molecular features for a given tumor location, we assessed whether this can be confirmed in late-onset colorectal cancer (LOCRC).  Methods: We analyzed clinical and molecular characteristics of LOCRC at different tumor locations in order to determine if there are differential phenotypes related with the location in the colon.  Results: Right colon cancers showed features associated with sporadic Microsatellite Instability: predominance of female cases and BRAF mutations, and an important mucinous component. Left colon cancers developed a higher number of polyps and multiple primary CRCs, showed the strongest familial component, and had better prognosis. Rectal cancers showed a predominantly sporadic phenotype, with worse prognosis and a CpG Island Me

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine gene expression availability
if '!Series_overall_design' in globals():
    overall_design = globals()['!Series_overall_design']
    if 'gene expression' in overall_design.lower() and 'Array Comparative Genomic Hybridisation (a-CGH)' not in overall_design:
        is_gene_available = True
    else:
        is_gene_available = False

# Determine availability of the variables and data type conversion functions

# Sample Characteristics Dictionary
sample_dict = globals().get('Sample Characteristics Dictionary', {})

# Gender determination
if 0 in sample_dict.keys():
    gender_row = 0

    def convert_gender(value):
        if 'Female' in value:
            return 0
        elif 'Male' in value:
            return 1
        else:
            return None

# Age determination
if 1 in sample_dict.keys():
    age_row = 1

    def convert_age(value):
        try:
            return int(value.split(':')[1].strip())
        except (ValueError, IndexError):
            return None

# Rectal Cancer determination
if 2 in sample_dict.keys():
    trait_values = sample_dict[2]
    # Check if there is significant variability to consider the variable
    if len(set(trait_values)) > 1:
        trait_row = 2

        def convert_trait(value):
            if 'MSS y CIMP-High' in value:
                return 1
            elif 'MSS y CIMP-Low/0' in value or 'MSI y CIMP-Low/0' in value or 'MSI y CIMP-High' in value:
                return 0
            else:
                return None

# Save cohort information
save_cohort_info('GSE108166', './preprocessed/Rectal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Rectal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Rectal_Cancer/trait_data/GSE108166.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Rectal_Cancer/cohort_info.json
